In [1]:
#Load CSV Data (Orders)
import pandas as pd

orders = pd.read_csv("orders.csv")
orders.head()



,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [2]:
#Load JSON Data (Users)
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [3]:
#Load SQL Data (Restaurants)
import sqlite3

conn = sqlite3.connect(":memory:")

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [4]:
#Merge the Data
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)
final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)

In [5]:
#Create Final Dataset
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [6]:
#Question:1
#Load the final merged dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

#Filter only Gold members
gold_df = df[df["membership"].str.lower() == "gold"]

#Calculate city-wise total revenue from Gold members
city_wise_revenue = (
    gold_df.groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

#Display result
print(city_wise_revenue)

#City with highest revenue
highest_revenue_city = city_wise_revenue.idxmax()
highest_revenue_amount = city_wise_revenue.max()

print("\nCity with highest Gold member revenue:")
print(highest_revenue_city, "->", highest_revenue_amount)

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

City with highest Gold member revenue:
Chennai -> 1080909.79


In [7]:
#Question:2
#Calculate average order value per cuisine
cuisine_aov = (
    df.groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

print(cuisine_aov)

#Cuisine with highest AOV
top_cuisine = cuisine_aov.idxmax()
top_aov = cuisine_aov.max()

print("\nCuisine with highest average order value:")
print(top_cuisine, "->", top_aov)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

Cuisine with highest average order value:
Mexican -> 808.0213444401395


In [8]:
#Question:3
#Calculate total order value per user
user_total_spend = (
    df.groupby("user_id")["total_amount"]
    .sum()
)

#Count users whose total spending is greater than ₹1000
high_value_users = user_total_spend[user_total_spend > 1000]

print("Number of users with total orders > ₹1000:", len(high_value_users))

Number of users with total orders > ₹1000: 2544


In [9]:
#Question:4
#Define rating ranges as per question
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

#Create rating range column
df["rating_range"] = pd.cut(
    df["rating"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

#Calculate total revenue per rating range
rating_range_revenue = (
    df.groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(rating_range_revenue)

rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64


In [10]:
#Question:5
#Filter Gold members
gold_df = df[df["membership"].str.lower() == "gold"]

#Compute city-wise average order value
city_aov_gold = (
    gold_df.groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

print(city_aov_gold)

#City with highest AOV
top_city = city_aov_gold.idxmax()
top_aov = city_aov_gold.max()

print("\nCity with highest AOV among Gold members:")
print(top_city, "->", round(top_aov, 2))

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

City with highest AOV among Gold members:
Chennai -> 808.46


In [12]:
#Question:6
#Count distinct restaurants per cuisine
restaurant_count = df.groupby("cuisine")["restaurant_name_y"].nunique()

#Calculate total revenue per cuisine
cuisine_revenue = df.groupby("cuisine")["total_amount"].sum()

#Combine results
summary = pd.DataFrame({
    "distinct_restaurants": restaurant_count,
    "total_revenue": cuisine_revenue
}).sort_values(by="distinct_restaurants")

print(summary)

         distinct_restaurants  total_revenue
cuisine                                     
Chinese                   120     1930504.65
Indian                    126     1971412.58
Italian                   126     2024203.80
Mexican                   128     2085503.09


In [13]:
#Question:7
#Total number of orders
total_orders = len(df)

# Orders placed by Gold members
gold_orders = len(df[df["membership"].str.lower() == "gold"])

# Percentage calculation
percentage_gold_orders = round((gold_orders / total_orders) * 100)

print("Percentage of orders by Gold members:", percentage_gold_orders, "%")

Percentage of orders by Gold members: 50 %


In [14]:
#Question:8
#Given restaurant options
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

#Filter only given restaurants
filtered_df = df[df["restaurant_name_y"].isin(options)]

#Calculate total orders and average order value
restaurant_stats = (
    filtered_df
    .groupby("restaurant_name_y")
    .agg(
        total_orders=("order_id", "count"),
        avg_order_value=("total_amount", "mean")
    )
)

#Apply condition: less than 20 orders
result = restaurant_stats[restaurant_stats["total_orders"] < 20]

#Get restaurant with highest average order value
final_answer = result.sort_values(
    by="avg_order_value",
    ascending=False
).head(1)

print(final_answer)

Empty DataFrame
Columns: [total_orders, avg_order_value]
Index: []


In [15]:
#Question:9
#Given combinations
options = [
    ("Gold", "Indian"),
    ("Gold", "Italian"),
    ("Regular", "Indian"),
    ("Regular", "Chinese")
]

#Calculate revenue for each combination
results = {}

for membership, cuisine in options:
    revenue = df[
        (df["membership"].str.lower() == membership.lower()) &
        (df["cuisine"] == cuisine)
    ]["total_amount"].sum()
    
    results[f"{membership} + {cuisine}"] = revenue

print(results)

{'Gold + Indian': np.float64(979312.31), 'Gold + Italian': np.float64(1005779.05), 'Regular + Indian': np.float64(992100.27), 'Regular + Chinese': np.float64(952790.9099999999)}


In [16]:
#Question:10
#Convert order_date to datetime
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)

#Extract quarter (1–4)
df["quarter"] = df["order_date"].dt.quarter

#Calculate total revenue per quarter
quarter_wise_revenue = (
    df.groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(quarter_wise_revenue)

quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64


In [17]:
#Question:11
#Count orders placed by Gold members
gold_orders = len(df[df["membership"].str.lower() == "gold"])

print("Total orders placed by Gold members:", gold_orders)

Total orders placed by Gold members: 4987


In [18]:
#Question:12
#Filter orders from Hyderabad city
hyderabad_orders = df[df["city"].str.lower() == "hyderabad"]

#Calculate total revenue (rounded)
total_revenue_hyderabad = round(hyderabad_orders["total_amount"].sum())

print("Total revenue from Hyderabad:", total_revenue_hyderabad)

Total revenue from Hyderabad: 1889367


In [19]:
#Question:13
#Count distinct users
distinct_users = df["user_id"].nunique()

print("Number of distinct users who placed at least one order:", distinct_users)

Number of distinct users who placed at least one order: 2883


In [20]:
#Question:14
#Filter Gold members
gold_orders = df[df["membership"].str.lower() == "gold"]

#Calculate average order value
avg_order_value_gold = round(gold_orders["total_amount"].mean(), 2)

print("Average order value for Gold members:", avg_order_value_gold)

Average order value for Gold members: 797.15


In [21]:
#Question:15
#Count orders where restaurant rating is >= 4.5
high_rating_orders = (df["rating"] >= 4.5).sum()

print("Number of orders for restaurants with rating ≥ 4.5:", high_rating_orders)

Number of orders for restaurants with rating ≥ 4.5: 3374


In [22]:
#Question:16
#Filter Gold members
gold_df = df[df["membership"].str.lower() == "gold"]

#Identify top revenue city among Gold members
top_city = (
    gold_df.groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

#Count orders in that city
orders_in_top_city = len(gold_df[gold_df["city"] == top_city])

print("Top revenue city (Gold members):", top_city)
print("Number of orders in that city:", orders_in_top_city)

Top revenue city (Gold members): Chennai
Number of orders in that city: 1337
